# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

Find the top 5 products from each category based on highest total sales. The output should be sorted by category in ascending order and by sales in descending order within each category, i.e. within each category product with highest margin should sit on the top. Save the query as top_five_products_each_category, containing the following columns:

- category
- product_name
- product_total_sales (rounded to two decimal places)
- product_total_profit (rounded to two decimal places)
- product_rank

In [ ]:
-- top_five_products_each_category
WITH RankedProducts AS (
    SELECT
        p.category,
        p.product_name,
        SUM(o.sales) AS product_total_sales,
        SUM(o.profit) AS product_total_profit,
        RANK() OVER (PARTITION BY p.category ORDER BY SUM(o.sales) DESC) AS product_rank
    FROM products AS p
    INNER JOIN orders AS o
        ON p.product_id = o.product_id
    GROUP BY p.category, p.product_name
)
-- Pega os 5 primeiros de cada grupo
SELECT *
FROM RankedProducts
WHERE product_rank <= 5
ORDER BY category, product_total_sales DESC;

Calculate the quantity for orders with missing values in the quantity column by determining the unit price for each product_id using available order data, considering relevant pricing factors such as discount, market, or region. Then, use this unit price to estimate the missing quantity values. The calculated values should be stored in the calculated_quantity column. Save query output as impute_missing_values, containing the following columns:

- product_id
- discount
- market
- region
- sales
- quantity
- calculated_quantity (rounded to zero decimal places)

In [ ]:
WITH average_unit_prices AS (
    SELECT
        product_id,
        discount,
        market,
        region,
        -- Calcula o preço unitário médio/ponderado
        SUM(sales) / SUM(quantity) AS estimated_unit_price
    FROM orders
    -- Filtra apenas os pedidos que têm dados completos e válidos
    WHERE quantity IS NOT NULL AND sales IS NOT NULL AND quantity > 0
    -- Agrupa para ter um preço por combinação de produto, desconto, mercado e região
    GROUP BY
        product_id,
        discount,
        market,
        region
),
missing_quantities AS (
    SELECT
        product_id,
        discount,
        market,
        region,
        sales,
        quantity -- Vai ser NULL aqui
    FROM orders
    WHERE quantity IS NULL
)
SELECT
    mq.product_id,
    mq.discount,
    mq.market,
    mq.region,
    mq.sales,
    mq.quantity,
    -- Converte a divisão para NUMERIC antes de arredondar
    ROUND(CAST(mq.sales / aup.estimated_unit_price AS NUMERIC), 0) AS calculated_quantity
FROM missing_quantities AS mq
INNER JOIN average_unit_prices AS aup
    ON mq.product_id = aup.product_id
    AND mq.discount = aup.discount
    AND mq.market = aup.market
    AND mq.region = aup.region
ORDER BY
    mq.product_id,
    mq.market,
    mq.region;